# Luna: parallel scans from notebook.

For running [Luna.jl](https://github.com/LupoLab/Luna.jl) jobs/[scans in parallel](http://lupo-lab.com/Luna.jl/stable/scans.html#Parameter-scans) from [Jupyter/IJulia notebooks](https://julialang.github.io/IJulia.jl/stable/).

**Instructions**

- Set all job params in first cell.
- Run all cells.
- All outputs, including figures, will be in generated directory (per scan).
- Final cell will also display images in the notebook, but may need to be re-run manually after parallel processes are complete (spawned as background jobs).


**What and why**

Jupyter is great, and I like to do everything in/from a notebook... but dispatching [Luna.jl](https://github.com/LupoLab/Luna.jl) jobs/[scans in parallel](http://lupo-lab.com/Luna.jl/stable/scans.html#Parameter-scans) from a notebook was buggy and inconsistent when I tried it. 

Instead of running jobs directly, this template writes a job file, then dispatches that to a parallel Julia pool via a shell script. Each set of results is put in a separate dir, along with output figures, for later inspection.


**Notes**

- To function, this requires a working [IJulia installation](https://julialang.github.io/IJulia.jl/stable/), and [Luna.jl](https://github.com/LupoLab/Luna.jl) - these can be installed via `using Pkg; Pkg.add("IJulia"); Pkg.add("Luna")`. For the final image display cell, you may also need `import Pkg; Pkg.add("Images"); Pkg.add("FileIO")`.
- The job template is lifted [more-or-less directly from the Luna.jl parameter scan docs](http://lupo-lab.com/Luna.jl/stable/scans.html#Parameter-scans), except with the addition of multi-mode handling (per [Luna.jl #316](https://github.com/LupoLab/Luna.jl/issues/316)).
- The parallel jobs are detached from the notebook, so should persist until completed, even at notebook close/disconnect.
- Tested in Julia v1.8.5, IJulia v1.24.0, Luna v0.2.0. 
- This notebook and scripts: https://github.com/phockett/Luna.jl-jupyterDispatch
- Additional notes at end of notebook.


## Define job and write to file from template

This is necessary since running parallel jobs directly from notebook (IJulia) seems to be buggy/inconsistent. Instead write job file and run as detached process.

In [ ]:
# Define job...

# Fixed parameters:
# For parameters see http://lupo-lab.com/Luna.jl/dev/interface.html
a = 450e-6   # Fixed radius
flength = 2.5
gas = :Ar
λ0 = 800e-9
τfwhm = 35e-15
λlims = (100e-9, 4e-6)
trange = 400e-15

# ADD MODES
modes = 2

#*** Scan dimensions:
energies = collect(range(50e-6, 500e-6; length=2))
pressures = collect(0.6:0.5:1.6)

#*** Job name and location - this assumes working dir is notebook dir
using Dates
jobStart = now()
dateString = Dates.format(now(), "ddmmyy")
scanName = "$(gas)_$(Int(a*1e6))mum_$(flength)m_m$(modes)_$(dateString)"

# @__DIR__ gives the directory of the current file
outputdir = joinpath(@__DIR__, "scanoutput_$scanName")
mkdir(outputdir)

print("Init job: $scanName")
print("\nScan dir: $outputdir")
print("\nTimestamp: $jobStart")

# Set parallel
# Requires ~1Gb per core, suggest <30 on Jake, depending on current base load (usually ~20-40Gb, of 64Gb)
# TODO: mem checks here
cores = 20

## Write template job and run

No need to edit the rest, just run cells below to generate job file and run.

In [ ]:
# Job file from template
job = """
using Luna
import PyPlot: plt

# Name
# scanName = "Ar_450mum_test_050223"

# Fixed parameters:
# For parameters see http://lupo-lab.com/Luna.jl/dev/interface.html
a = $a   # Fixed radius
flength = $flength
gas = :$gas
λ0 = $λ0
τfwhm = $τfwhm
λlims = $λlims
trange = $trange

# ADD MODES
modes = $modes

# Name
# using Dates
# dateString = Dates.format(now(), "ddmmyy")
scanName = "$scanName"
outputdir = "$outputdir"
datadir = joinpath(outputdir, "data")

# @__DIR__ gives the directory of the current file - NOW PASSED ABOVE
# outputdir = joinpath(@__DIR__, "scanoutput_$scanName")

#*** Scan dimensions:
energies = $energies
pressures = $pressures

# scan variables can be passed directly to the Scan constructor...
scan = Scan(scanName; energy=energies)
#...or added later
addvariable!(scan, :pressure, pressures)

runscan(scan) do scanidx, energy, pressure
    prop_capillary(a, flength, gas, pressure; λ0, τfwhm, energy, modes,
                   λlims, trange, scan, scanidx, filepath=datadir)
end

"""

# Add processing - use raw to avoid interp here
proc=raw"""

#*** Processing
print("\n*** Processing from $datadir...")

# CASE WITH SUM OVER MODES (QUITE SLOW)
λ, Iλ, zstat, edens, max_peakpower = Processing.scanproc(datadir) do output
#     size(output)
    λ, Iλ = Processing.getIω(output, :λ)
    zstat = Processing.VarLength(output["stats"]["z"])
    edens = Processing.VarLength(output["stats"]["electrondensity"])
    max_peakpower = maximum(output["stats"]["peakpower"])
#     Processing.Common(λ), Iλ[:, end], zstat, edens, max_peakpower
    Processing.Common(λ), dropdims(sum(Iλ[:, :, end]; dims=2); dims=2), zstat, edens, max_peakpower
end

# Plot
fig, axs = plt.subplots(3, round(Int,length(pressures)/3))
fig.set_size_inches(20, 14)
for (pidx, pressure) in enumerate(pressures)
    ax = axs[pidx]
    global img = ax.pcolormesh(λ*1e9, energies*1e6, 10*Maths.log10_norm(Iλ[:, :, pidx])')
    img.set_clim(-40, 0)
    ax.set_xlabel("Wavelength (nm)")
    ax.set_ylabel("Energy (μJ)")
    ax.set_title("Pressure: $pressure bar")
    ax.set_xlim(100, 1200)
end
plt.colorbar(img, ax=axs, label="Energy density (dB)")

figName = joinpath(outputdir,scanName*"_sum.png")
plt.savefig(figName, dpi=300, format="png")

print("\nWrote figure $figName")


#*** For individual modes
# Quick hack - loop over processing and plotting per mode, but should be able to consolidate this.

if modes > 1
    for mode = 1:modes
        print("\nProcessing mode $mode.")

        λ, Iλ, zstat, edens, max_peakpower = Processing.scanproc(datadir) do output
        #     size(output)
            λ, Iλ = Processing.getIω(output, :λ)
            zstat = Processing.VarLength(output["stats"]["z"])
            edens = Processing.VarLength(output["stats"]["electrondensity"])
            max_peakpower = maximum(output["stats"]["peakpower"])
        #     Processing.Common(λ), Iλ[:, end], zstat, edens, max_peakpower
            Processing.Common(λ), Iλ[:, mode, end], zstat, edens, max_peakpower
        end


        fig, axs = plt.subplots(3, round(Int,length(pressures)/3))
        fig.set_size_inches(20, 14)
        for (pidx, pressure) in enumerate(pressures)
            ax = axs[pidx]
            global img = ax.pcolormesh(λ*1e9, energies*1e6, 10*Maths.log10_norm(Iλ[:, :, pidx])')
            img.set_clim(-40, 0)
            ax.set_xlabel("Wavelength (nm)")
            ax.set_ylabel("Energy (μJ)")
            ax.set_title("Pressure: $pressure bar")
            ax.set_xlim(100, 1200)
        end
        plt.colorbar(img, ax=axs, label="Energy density (dB)")

        figName = joinpath(outputdir,scanName*"_m$mode.png")
        plt.savefig(figName, dpi=300, format="png")

        print("\nWrote figure $figName")

    end
end

"""



# Write to file
jobFileName = joinpath(outputdir,scanName*".jl")
jobFile = open(jobFileName,"w")
write(jobFile, job * proc)
close(jobFile)

print("Written job to $jobFileName")

## Run job

IF THIS GOES BAD: `sudo pkill -f "julia"` from terminal may help.

Modify cmd to full path to `lunaParallel.sh` if required (assumes working dir currently).

In [ ]:
# Detached run with log file
# Run via bash script
# Note: this should persist even if notebook closed, but not yet well tested.
fullPath = joinpath(outputdir,scanName)
cmd = `./lunaParallel.sh $fullPath "$cores"`

run(cmd)

# Basic run - displays in notebook, no detach
# cmd = `julia $scanName.jl -q -p 10`
# run(cmd)

## Display results

NOTE: this won't show anything until after parallel runs completed.

USE Image library for this, run `import Pkg; Pkg.add("Images"); Pkg.add("FileIO")` in notebook or at CLI to install.

DOCS: https://juliaimages.org/latest/

TODO: larger/rescaled images here? No idea how to do that. May be able to use PyPlot, but `plt.imread` just returns matrix (data), not full plot? See https://juliapackages.com/p/pyplot for more details.

In [ ]:
using Images, FileIO

# Set PNG search func, https://stackoverflow.com/a/20485113
searchdir(path,key) = filter(x->occursin(key,x), readdir(path))
imageNames = searchdir(outputdir,"png") 

# loop over img files, load (with Images, see https://juliaimages.org) and display
for imgName in imageNames

    # specify the path to your local image file
    img_path = joinpath(outputdir,imgName)
    img = load(img_path)
    
    print(imgName)
    display(img)
    
end

## Env

In [1]:
using Pkg
Pkg.status()

Status `~/.julia/environments/v1.8/Project.toml`
  [5789e2e9] FileIO v1.16.0
  [7073ff75] IJulia v1.24.0
  [916415d5] Images v0.25.2
  [30eb0fb0] Luna v0.2.0
  [d330b81b] PyPlot v2.11.0


---

**Version notes**

23/02/23

v4, tidied up and pushed to Github, https://github.com/phockett/Luna.jl-jupyterDispatch.


20/02/23

Added quick image display routine. Needs work, but functional.


19/02/23

Now working with shell script to run, includes all-mode processing. One dir per scan.

TODO: pull plots (images) back into notebook.


16/02/23

Quick attempt at using notebook to define job > save text to file > trigger parallel job in background with Julia.

(Alternative might be to use nbconvert to run notebook...?)

Status: basically working, few things could be neater/automated.